In [ ]:
%pylab inline
%matplotlib notebook

# Goal

Our goal is to extend the one dimensional problem of the previous notebook, to dimensions 2 and 3. 

# Collections of one dimensional Lagrange basis functions

$$
l_i(x) := \prod_{j=0, j\neq i}^{n} \frac{x-x_j}{x_i-x_j}.
$$

In [ ]:
from numpy.polynomial.polynomial import Polynomial
from numpy.testing import *

We condisder a `dim`-dimensional problem. We setup a one dimensional set of basis finctions along each of `dim` direction. The first two lines in the previous notebook:
```
n = 6
q = linspace(0,1,n)
```
are substituted with two lists:
$$
\mathtt{N = [} \mathtt{n}_1,\mathtt{n}_2,\ldots, \mathtt{n}_{\mathtt{dim}}\mathtt{]}\\
\mathtt{Q = [[} \mathtt{q}_0,\ldots \mathtt{q}_{\mathtt{n}_1}\mathtt{]}\ldots\mathtt{[}\mathtt{q}_0,\ldots \mathtt{q}_{\mathtt{n}_{\mathtt{dim}}}\mathtt{]}\mathtt{]}
$$

In [ ]:
dim = 2
N = [6,7] # same size as dim
assert_equal(dim,len(N))
Q = [linspace(0,1,n) for n in N]

The list `Q` is automatically generated by the program with the correct dimensionality. `dim`, and `N` can be seen as input parameters that the user has to set in a coherent way. Tho check that we equip this section of the program with an *assertion*. An assertion is an useful that throws an error to defend the program to possible mistakes. The line:
```
assert_equal(dim,len(N))
```
throws an error in case the length of `N` does not match `dim`.

In [ ]:
print(Q[0])

We prepare a list `LL` we expect it to have `len(LL) = dim`. For each dimension we populate it with the set of basis functions along the corresponding dimension.

In [ ]:
LL = []
for d in range(dim):
    n = N[d]
    q = Q[d]
    LL += [[Polynomial.fromroots([xj for xj in q if xj != q[i]]) for i in range(n)]]

Only for the brave:
```
LL = []
for n,q in zip(N,Q):
    LL += [[Polynomial.fromroots([xj for xj in q if xj != q[i]]) for i in range(n)]]
```

In [ ]:
for d in range(dim):
    n = N[d]
    q = Q[d]
    L = LL[d]
    LL[d] = [L[i]/L[i](q[i]) for i in range(n)]

Only for the brave:
```
for n,q,L in zip(N,Q,LL):
    LL[d] = [L[i]/L[i](q[i]) for i in range(n)]
```

It is good idea to prepare lists for first and second derivatives of the basis functions. Depending on the problem formulation we will use the appropriate ones.

In [ ]:
LL_prime = []
for d in range(dim):
    n = N[d]
    q = Q[d]
    L = LL[d]
    LL_prime += [[L[i].deriv() for i in range(n)]]

The brave version:
```
for L,n in zip(LL,N):
    LL_prime += [[L[i].deriv() for i in range(n)]]
```

In [ ]:
LL_prime_prime = []
for d in range(dim):
    n = N[d]
    q = Q[d]
    L_prime = LL_prime[d]
    LL_prime_prime += [[L_prime[i].deriv() for i in range(n)]]

Try your own *brave* version...

In [ ]:
x = linspace(0,1,1025)
for d in range(dim):
    L = LL[d]
    q = Q[d]
    n = N[d]
    figure()
    for f in L:
        plot(x, f(x))
    plot(q, 0*q, 'ro')
    show()

# Two dimensional problems

Let's start by understanding **Tensor Product** structures

In [ ]:
x = linspace(0,1,101)
y = linspace(0,1,201)

Let's define a **scalar function of two variables**:

In [ ]:
g = lambda x,y: sin(2*pi*x)*cos(2*pi*y)

If you want to evaluate this function on **x** and **y**, for all combinations of the various x and y, then you have to make sure you **broadcast** them to have the same size. The following does exactly this:

In [ ]:
Ge = zeros((len(y), len(x)))
for i in xrange(len(x)):
    for j in xrange(len(y)):
        Ge[j,i] = g(x[i], y[j])

In [ ]:
X, Y = meshgrid(x,y) 
Gnumpy = g(X,Y)

error = Gnumpy-Ge
norm(error)

## Plotting 3d plots

A quick search on google gives us the following option:

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = pyplot.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X,Y,g(X,Y))

# Einstein notation in numpy

As python is a powerful language, especially in resembling the mathematical language, we expect its libraries to have powerful tools. We recommend the user to check [`einsum`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.einsum.html#numpy.einsum) documentation. Here we limit the discussion to the most meaningful examples.

In [ ]:
a = arange(25).reshape(5,5)
b = arange(5)

In [ ]:
print einsum('ij', a)
print einsum('ji', a)

In [ ]:
print einsum('ij,j', a, b)
print np.dot(a, b)

In [ ]:
z = arange(5)
w = arange(6)
print einsum('i...->i', z)
print einsum('i,j', z,w)
print einsum('i,j -> ij', z,w)
print einsum('i,j -> ji', z,w)

Use the `einsum` command for a smarter way to plot tensor product functions, remember that tensor product functions are of the form `g1(x)*g2(y)`

In [ ]:
fig = pyplot.figure()
ax = fig.add_subplot(111, projection='3d')

F = einsum('i,j -> ji', sin(2*pi*x), cos(2*pi*y))

ax.plot_surface(X,Y,F)

Try plotting $g(x,y) := l_3(x) l_4(y)$

In [ ]:
fig = pyplot.figure()
ax = fig.add_subplot(111, projection='3d')

F = einsum('i,j -> ji', LL[0][3](x), LL[1][4](y))

ax.plot_surface(X,Y,F)

# A two dimensional problem (without bc!)

Consider the following problem
$$
-\Delta u + u = f(s) \text{ in } [0,1]\times[0,1]
$$

And solve it using a tensor product vector space:

$$
V_h := \text{span} \{v_{ij}\}_{i,j = 0}^{n}, \qquad v_{ij}(\mathbf{x}) := l_i(x_1)l_j(x_2)
$$

a function in $V_h$ can be seen as a linear combination of $n_1\times n_2$ coeffients times the basis functions, and can be "unrolled" in a vector of dimension $1\times n_1 n_2$ or $n_1n_2 \times 1$ by a simple reshape, or by a `flatten` operation.

$$
u_h(x) := u^{ij}v_{ij} (x)
$$

and the Laplacian of $v_{ij}$ can be computed simply by:
$$
\Delta v_{ij}(x) := l_i''(x_1)l_j''(x_2)
$$

Now let's use $a,b,c,d$ for indices running over **collocation points** and $i,j,k,l,m$ for indices running over basis function indices.


Start by computing 
$$
A_{ib} := l_i(q_b)
$$
$$
B_{ib} := l'_i(q_b)
$$
$$
C_{ib} := l''_i(q_b)
$$